In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pyarrow.dataset as ds
import duckdb

pd.set_option('display.max_columns', None)

# Open-Meteo Forecasts

## Partition data on `["city", "year_month"]`

In [ ]:
df = pd.read_parquet(
    "data/aggregated/aggregated_multi_model_forecasts.parquet"
)

In [ ]:
df.head()

In [ ]:
from src.data.utilities import *

# df = normalize_field_names(df)
# df = add_date_fields(df)
df = convert_datetime_to_utc(df)

In [ ]:
df.head()

In [ ]:
df.to_parquet("data/local_data/forecasts", engine="pyarrow", partition_cols=["city", "year_month"])

In [ ]:
del df

# Accuweather Forecasts

In [3]:
accuweather_current_path = "/Users/luketownsend/Desktop/projects/tetlock/data_collection/..data/accuweather_raw/"
accuweather_path_1 = "/Users/luketownsend/Desktop/projects/tetlock/data_collection/data/accuweather_raw/archive/"
accuweather_path_2 = "/Users/luketownsend/Desktop/projects/tetlock/data_collection/data/accuweather_raw/archive/archive/"

In [ ]:
import pyarrow.dataset as ds

dataset = ds.dataset(accuweather_path_1, format="parquet")
table = dataset.to_table()
df = table.to_pandas()  # optional

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
dataset = ds.dataset(accuweather_path_2, format="parquet")
table = dataset.to_table()
df_2 = table.to_pandas()  # optional

In [ ]:
df_2.shape

In [ ]:
accuweather_current_path

In [5]:
dataset = ds.dataset(accuweather_current_path, format="parquet")

num_files = len(dataset.files)
print(num_files)

table = dataset.to_table()

61039


In [7]:
import pyarrow as pa
import pyarrow.parquet as pq

file_paths = [frag.path for frag in dataset.get_fragments()]

pq.write_table(pa.Table.from_pydict({"file_path": file_paths}), "file_list.parquet")
pq.write_table(table, "data/local_data/accuweather/current_file_compacted.parquet")

In [8]:
df_3 = table.to_pandas()  # optional

In [9]:
df_3.shape

(732468, 84)

In [10]:
df_3.head()

,datetime,EpochDateTime,WeatherIcon,phrase,has_precip,IsDaylight,rh,IndoorRelativeHumidity,uv_index,UVIndexFloat,UVIndexText,precip_prob,ThunderstormProbability,RainProbability,SnowProbability,IceProbability,CloudCover,AccuLumenBrightnessIndex,MobileLink,Link,temp,temp_unit,Temperature.UnitType,realfeel,realfeel_unit,RealFeelTemperature.UnitType,RealFeelTemperature.Phrase,RealFeelTemperatureShade.Value,RealFeelTemperatureShade.Unit,RealFeelTemperatureShade.UnitType,RealFeelTemperatureShade.Phrase,WetBulbTemperature.Value,WetBulbTemperature.Unit,WetBulbTemperature.UnitType,WetBulbGlobeTemperature.Value,WetBulbGlobeTemperature.Unit,WetBulbGlobeTemperature.UnitType,DewPoint.Value,DewPoint.Unit,DewPoint.UnitType,wind_speed,wind_unit,Wind.Speed.UnitType,Wind.Direction.Degrees,wind_dir,Wind.Direction.English,WindGust.Speed.Value,WindGust.Speed.Unit,WindGust.Speed.UnitType,Visibility.Value,Visibility.Unit,Visibility.UnitType,Ceiling.Value,Ceiling.Unit,Ceiling.UnitType,TotalLiquid.Value,TotalLiquid.Unit,TotalLiquid.UnitType,Rain.Value,Rain.Unit,Rain.UnitType,Snow.Value,Snow.Unit,Snow.UnitType,Ice.Value,Ice.Unit,Ice.UnitType,Evapotranspiration.Value,Evapotranspiration.Unit,Evapotranspiration.UnitType,SolarIrradiance.Value,SolarIrradiance.Unit,SolarIrradiance.UnitType,forecast_temperature,city,model_timestamp,model_name,model_id,latitude,longitude,series_id,nws_id,tz_convert,weather_id
0,2025-09-11 22:00:00-05:00,1757646000,34,Mostly clear,False,False,66,66,0,0.0,Low,0,0,0,0,0,11,0.0,http://www.accuweather.com/en/us/austin-tx/787...,http://www.accuweather.com/en/us/austin-tx/787...,85.0,F,18,91.0,F,18,Hot,91.0,F,18,Hot,77.0,F,18,83.0,F,18,73.0,F,18,6.9,mi/h,9,158,SSE,SSE,13.8,mi/h,9,10.0,mi,2,30000.0,ft,0,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,W/m²,33,85.0,Austin,2025-09-12 02:03:39.926390+00:00,accuweather_12h,1001,30.1941,-97.6711,KXHIGHAUS,AUS,America/Chicago,05a7e7f73567bcc0ce8fac95250c4a0b26580d12fa0797...
1,2025-09-11 23:00:00-05:00,1757649600,34,Mostly clear,False,False,66,66,0,0.0,Low,0,0,0,0,0,11,0.0,http://www.accuweather.com/en/us/austin-tx/787...,http://www.accuweather.com/en/us/austin-tx/787...,83.0,F,18,86.0,F,18,Very Warm,86.0,F,18,Very Warm,74.0,F,18,80.0,F,18,70.0,F,18,6.9,mi/h,9,167,SSE,SSE,13.8,mi/h,9,10.0,mi,2,30000.0,ft,0,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,W/m²,33,83.0,Austin,2025-09-12 02:03:39.926390+00:00,accuweather_12h,1001,30.1941,-97.6711,KXHIGHAUS,AUS,America/Chicago,05a7e7f73567bcc0ce8fac95250c4a0b26580d12fa0797...
2,2025-09-12 00:00:00-05:00,1757653200,34,Mostly clear,False,False,74,74,0,0.0,Low,0,0,0,0,0,11,0.0,http://www.accuweather.com/en/us/austin-tx/787...,http://www.accuweather.com/en/us/austin-tx/787...,80.0,F,18,84.0,F,18,Very Warm,84.0,F,18,Very Warm,74.0,F,18,79.0,F,18,71.0,F,18,5.8,mi/h,9,180,S,S,13.8,mi/h,9,10.0,mi,2,30000.0,ft,0,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,W/m²,33,80.0,Austin,2025-09-12 02:03:39.926390+00:00,accuweather_12h,1001,30.1941,-97.6711,KXHIGHAUS,AUS,America/Chicago,05a7e7f73567bcc0ce8fac95250c4a0b26580d12fa0797...
3,2025-09-12 01:00:00-05:00,1757656800,34,Mostly clear,False,False,80,80,0,0.0,Low,0,0,0,0,0,11,0.0,http://www.accuweather.com/en/us/austin-tx/787...,http://www.accuweather.com/en/us/austin-tx/787...,78.0,F,18,82.0,F,18,Very Warm,82.0,F,18,Very Warm,74.0,F,18,78.0,F,18,71.0,F,18,5.8,mi/h,9,191,S,S,13.8,mi/h,9,10.0,mi,2,30000.0,ft,0,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,W/m²,33,78.0,Austin,2025-09-12 02:03:39.926390+00:00,accuweather_12h,1001,30.1941,-97.6711,KXHIGHAUS,AUS,America/Chicago,05a7e7f73567bcc0ce8fac95250c4a0b26580d12fa0797...
4,2025-09-12 02:00:00-05:00,1757660400,34,Mostly clear,False,False,84,84,0,0.0,Low,0,0,0,0,0,11,0.0,http://www.accuweather.com/en/us/austin-tx/787...,http://www.accuweather.com/en/us/austin-tx/787...,76.0,F,18,81.0,F,18,Pleasant,81.0,F,18,Pleasant,73.0,F,18,77.0,F,18,71.0,F,18,4.6,mi/h,9,197,SSW,SSW,12.7,mi/h,9,10.0,mi,2,30000.0,ft,0,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,in,1,0.0,W/m²

In [11]:
len(file_paths)

61039

In [ ]:
df_full = pd.concat([df, df_2, df_3], ignore_index=True)

In [12]:
def dedupe_accuweather_data(df: pd.DataFrame) -> pd.DataFrame:
    dedup_cols = [c for c in df.columns if c != "model_timestamp"]

    # 4. Sort by timestamp so earliest comes first, then drop duplicates
    df_dedup = (
        df
        .sort_values("model_timestamp")  # earliest first
        .drop_duplicates(subset=dedup_cols, keep="first")
    )
    return df_dedup

In [15]:
df_3.shape

(732468, 84)

In [13]:
deduped_df = dedupe_accuweather_data(df_3)

In [14]:
deduped_df.shape

(71078, 84)

In [16]:
del df_3

In [ ]:
# Source: One large parquet file
# Read Approach: read full file, then filter

df = pd.read_parquet(
    "data/aggregated/aggregated_multi_model_forecasts.parquet"
)
# 35.8s to read

df[(df["city"] == "Austin") & (df["year_month"] == "2025-10")]
# 6.1s to filter

# 41.9s total

In [ ]:
# Source: One large parquet file
# Read Approach: pd.read_parquet() w/ filters & engine='pyarrow'

df = pd.read_parquet(
    "data/aggregated/aggregated_multi_model_forecasts.parquet",
    engine="pyarrow",
    filters=[("city", "==", "Austin"), ("year_month", "==", "2025-10")],
    columns=["datetime", "forecast_temperature", "model_name"]
)

# Read in 3.7s, 3.3s, 3.1s

In [ ]:
dataset = ds.dataset("data/local_data/forecasts", format="parquet", partitioning="hive")

table = dataset.to_table(
    filter=(
        (ds.field("city") == "Austin")
        & (ds.field("year_month") == "2025-10")
    ),
    columns=["datetime", "forecast_temperature", "model_name"]
)

df = table.to_pandas()

#0.1s, 0.1s, 0.1s

# NWS Observations


In [ ]:
df = pd.read_parquet("data/aggregated/aggregated_nws_observations.parquet")

In [ ]:
from src.data.utilities import *

df = normalize_field_names(df)
df = add_date_fields(df)
df = convert_datetime_to_utc(df)

In [ ]:
df

## Partion on `["city", "year_month"]`

In [ ]:
df.to_parquet("data/local_data/nws_observations", engine="pyarrow", partition_cols=["city", "year_month"])

In [ ]:
del df

# Markets Data Processed

In [ ]:
df = pd.read_parquet("data/aggregated/aggregated_markets_data_processed.parquet")

In [ ]:
df.columns

In [ ]:
df = pd.read_parquet("data/aggregated/aggregated_winning_markets_data.parquet")

In [ ]:
df

## Testing Various Approaches to Read Parquet Files: